In [1]:
!nvidia-smi

Tue Aug 13 17:45:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
! pip install diffusers transformers ftfy  -q

In [2]:
! pip install gradio==3.50.2

  Using cached gradio-3.50.2-py3-none-any.whl.metadata (17 kB)
  Using cached gradio_client-0.6.1-py3-none-any.whl.metadata (7.1 kB)
Using cached gradio-3.50.2-py3-none-any.whl (20.3 MB)
Using cached gradio_client-0.6.1-py3-none-any.whl (299 kB)
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.3.0
    Uninstalling gradio_client-1.3.0:
      Successfully uninstalled gradio_client-1.3.0
  Attempting uninstall: gradio
    Found existing installation: gradio 4.41.0
    Uninstalling gradio-4.41.0:
      Successfully uninstalled gradio-4.41.0


In [4]:
from huggingface_hub import notebook_login

notebook_login()

/opt/conda/envs/ipenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [4]:
from io import BytesIO
from torch import autocast
import requests
import PIL
import torch
from diffusers import DiffusionPipeline
import gradio as gr


device = "cuda"
pipe = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    #"CompVis/stable-diffusion-v1-4",
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)

def predict(img_dict, prompt):
  init_img =  img_dict['image'].convert("RGB").resize((512, 512))
  mask_img = img_dict['mask'].convert("RGB").resize((512, 512))

  output = pipe(prompt=prompt, image=init_img, mask_image=mask_img, strength=0.75)

  return(output.images[0])


gr.Interface(
    predict,
    title = 'FreshFits by Pixlr (Stable Diffusion In-Painting Tool) by Kevin',
    description='Use this tool to look your best copping the freshest fits. You can edit images by uploading and sketching on the area to revamp, then the software generates new images based on your prompt.',
    inputs=[
        gr.Image(label='Upload or Edit Image',source = 'upload', tool = 'sketch', type = 'pil'),
        gr.Textbox(label='Outfit Description')
    ],
    outputs = [
        gr.Image(label='Generated Image')
        ]
).launch(share=True)


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/opt/conda/envs/ipenv/lib/python3.12/site-packages/diffusers/pipelines/pipeline_loading_utils.py:219: FutureWarning: You are loading the variant fp16 from runwayml/stable-diffusion-inpainting via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(
unet/diffusion_pytorch_model.safetensors not found
Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionInpaintPipeline and will be ignored.
Loading pipeline components...:  14%|█▍        | 1/7 [00:00<00:00,  7.86it/s]/opt/conda/envs/ipenv/lib/python3.12/site-p

Running on local URL:  http://127.0.0.1:7862
IMPORTANT: You are using gradio version 3.50.2, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://78ef8d16aade1fbe4b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


100%|██████████| 37/37 [00:05<00:00,  7.04it/s]
